In [8]:
import sys
sys.path.append('code')

In [146]:
import pandas as pd
import os
import numpy as np
from bokeh.layouts import gridplot
import test_jonne as tj

url_2 = "https://raw.githubusercontent.com/lukes/ISO-3166-Countries-with-Regional-Codes/master/all/all.csv"
regional_file_name = "datasets/regions/regions.csv"

file_name = 'datasets/data/WFPVAM_FoodPrices_version4_Retail.csv'

df = pd.read_csv(file_name)

# Each product in a graph for each country.

region_df = pd.read_csv(regional_file_name)
region_df.rename(columns={'name': 'adm0_name'}, inplace=True)
new_regions = region_df.loc[:, ['adm0_name', 'sub-region']]

df_regions = pd.merge(df, new_regions, on='adm0_name', how='left')
df = df_regions.copy()

# Add region and datetime cols.
df2 = df.copy()
df['date'] = pd.to_datetime(df.date, format='%Y-%m')

who = "datasets/global_mortality_who.csv"

mortality = pd.read_csv(who, skiprows=1)
mortality['date'] = pd.to_datetime(mortality['Year'], format='%Y')

print(mortality.columns)

df = pd.merge(df, mortality, left_on=['adm0_name', 'date'], right_on=['Country', 'date'])
df.drop(columns=['Country', 'Year'], inplace=True)
df = df.rename(columns={' Both sexes': 'mortality_sum', ' Male': 'mortality_male', ' Female': 'mortality_female'})

Index(['Country', 'Year', ' Both sexes', ' Male', ' Female', 'date'], dtype='object')


In [147]:
ref = pd.read_csv('server/python_flask/model/refugees.csv')
ref['datetime'] = pd.to_datetime(ref.date, format='%Y-%m-%d')
ref.drop(columns=['Unnamed: 0'], inplace=True)

refugees_sum = ref.groupby(['origin', 'datetime']).sum().reset_index()
df = pd.merge(df, refugees_sum, left_on=['adm0_name', 'date'], right_on=['origin', 'datetime'])



In [148]:
# MERGE STUFF
old_v1 = pd.read_csv('datasets/data/WFPVAM_FoodPrices_version1.csv')
all_currencies = pd.read_csv('datasets/currencies/all_currencies.csv')
all_currencies.columns = ['cur_name', 'date', 'rate']

old_v1 = pd.merge(old_v1, all_currencies, on=['cur_name', 'date'], how='left')
old_v1['date'] = pd.to_datetime(old_v1.date, format='%Y-%m')

df = pd.merge(df, old_v1[['adm0_name', 'mkt_name', 'cm_name', 'date', 'rate']], on=['adm0_name', 'mkt_name', 'cm_name', 'date'], how='left')

In [149]:
# GDP

gdp = pd.read_csv('datasets/GDP/GDP_per_capita.csv')
gdp = gdp.melt(id_vars=['country'])
gdp.columns = [tj.COUNTRY, tj.YEAR, 'GDP']
gdp[tj.YEAR] = gdp[tj.YEAR].astype(int)

# gdp['date'] = pd.to_datetime(gdp['mp_year'], format='%Y')

df[tj.YEAR] = df['date'].dt.year
df[tj.YEAR] = df[tj.YEAR].astype(int)
df = pd.merge(df, gdp[['adm0_name', tj.YEAR, 'GDP']], on=['adm0_name', tj.YEAR], how='left')
df.drop(columns=['mp_year'], inplace=True)

In [150]:
print(df.columns)



Index(['adm0_name', 'mkt_name', 'cm_name', 'mp_price', 'date', 'um_name',
       'Gradient', 'sub-region', 'mortality_sum', 'mortality_male',
       'mortality_female', 'origin', 'datetime', 'frequency', 'rate', 'GDP'],
      dtype='object')


In [139]:

# only_bread.plot(x='date', y='Diff')

def datetime(x):
    return np.array(x, dtype=np.datetime64)


from bokeh.layouts import gridplot
from bokeh.plotting import figure, show, output_file
from bokeh.models import HoverTool


# COLORS
from bokeh.palettes import Dark2_5 as palette
import itertools


colors = itertools.cycle(palette)

Index(['adm0_name', 'mkt_name', 'cm_name', 'mp_price', 'date', 'um_name',
       'Gradient', 'sub-region', 'mortality_sum', 'mortality_male',
       'mortality_female', 'origin', 'datetime', 'frequency', 'rate',
       'mp_year', 'GDP'],
      dtype='object')


# Question 1

Are there any food prices that are show negative/positive correlation, and is this correlation present throughout the years, or perhaps only in certain period? Can you perhaps detect possible ingredients of a certain other food product?



In [87]:
# Each product in a graph for each country.

country = 'Uganda'
y_axis = 'Gradient'

df_country = df.loc[df.adm0_name == country]
plots = []
country_groups = df_country.groupby(['mkt_name'])

# allow for interactive hover tool
# Import the ColumnDataSource
from bokeh.models import ColumnDataSource

for country_group, country_row in country_groups:
    plot = figure(x_axis_type="datetime", title="Products in {}".format(country_group), tools='hover,pan,wheel_zoom,box_zoom,reset')
    product_groups = country_row.groupby(['cm_name'])
    for (group, row), color in zip(product_groups, colors):
        # create CDS to enable dynamic hovering
        row_cds = ColumnDataSource(row)
        # set source to CDS
        # OK to use Gradient for derivative?
        plot.line('date', y_axis, color=color, legend=group, source=row_cds)
    plot.legend.click_policy = "hide"
    plot.legend.location = "top_right"
    hover = plot.select(dict(type=HoverTool))
    # set preffered info for hovertool to show
    hover.tooltips = [
        # need to fix daytime again now....
        ("date", "@date"),
        ("product", "@cm_name"),
        ("Gradient", "@Gradient")
        ]
    
    plots.append(plot)

show(gridplot([plots]))

W-1004 (BOTH_CHILD_AND_ROOT): Models should not be a document root if they are in a layout box: Figure(id='c302fd90-07b6-44e3-a8b6-9aef96390bca', ...)
W-1004 (BOTH_CHILD_AND_ROOT): Models should not be a document root if they are in a layout box: Figure(id='84dc665e-d409-48d7-9c17-eaaadb5f885a', ...)
W-1004 (BOTH_CHILD_AND_ROOT): Models should not be a document root if they are in a layout box: Figure(id='d65c1eef-3615-451a-983d-b2f3446e3025', ...)
W-1004 (BOTH_CHILD_AND_ROOT): Models should not be a document root if they are in a layout box: Figure(id='4f251dc9-42a3-43cf-9959-af06ae93803f', ...)
W-1004 (BOTH_CHILD_AND_ROOT): Models should not be a document root if they are in a layout box: Figure(id='de0cd80d-1fbb-4fa2-a8ee-1bb3dff09255', ...)
W-1004 (BOTH_CHILD_AND_ROOT): Models should not be a document root if they are in a layout box: Figure(id='b4959177-0795-4f96-91e9-e6c0d00a1e6c', ...)
W-1004 (BOTH_CHILD_AND_ROOT): Models should not be a document root if they are in a layout box

# Question 2
Do countries in similar regions, also show similar price differences? And if differences occur, can you find a potential explanation?

In [88]:
y_axis = 'Gradient'


plots = []
product_groups = df.groupby(['sub-region', 'cm_name', 'date'])[y_axis].mean().reset_index()
group = product_groups.groupby('sub-region')

for gr1, row1 in group:
    if gr1 == 'Southern Asia':
        plot = figure(x_axis_label="Date", y_axis_label = "Price", x_axis_type="datetime", title="Avg Products in {}".format(gr1), toolbar_location="below", tools='hover,pan,wheel_zoom,box_zoom,reset', plot_width=2800, plot_height=2800)
        product_groups = row1.groupby(['cm_name'])
        for (gr2, row2), color in zip(product_groups, colors):
            row2_cds = ColumnDataSource(row2)
            plot.line('date', y_axis, color=color, legend=gr2, source=row2_cds)
        plot.legend.click_policy = "hide"
        plot.legend.location = "top_right"
        hover = plot.select(dict(type=HoverTool))
        hover.tooltips = [
            ("Product", "@cm_name"),
            ("Value", "@Gradient"),
            ]

        plots.append(plot)
show(gridplot([plots]))

W-1004 (BOTH_CHILD_AND_ROOT): Models should not be a document root if they are in a layout box: Figure(id='c302fd90-07b6-44e3-a8b6-9aef96390bca', ...)
W-1004 (BOTH_CHILD_AND_ROOT): Models should not be a document root if they are in a layout box: Figure(id='84dc665e-d409-48d7-9c17-eaaadb5f885a', ...)
W-1004 (BOTH_CHILD_AND_ROOT): Models should not be a document root if they are in a layout box: Figure(id='d65c1eef-3615-451a-983d-b2f3446e3025', ...)
W-1004 (BOTH_CHILD_AND_ROOT): Models should not be a document root if they are in a layout box: Figure(id='4f251dc9-42a3-43cf-9959-af06ae93803f', ...)
W-1004 (BOTH_CHILD_AND_ROOT): Models should not be a document root if they are in a layout box: Figure(id='de0cd80d-1fbb-4fa2-a8ee-1bb3dff09255', ...)
W-1004 (BOTH_CHILD_AND_ROOT): Models should not be a document root if they are in a layout box: Figure(id='b4959177-0795-4f96-91e9-e6c0d00a1e6c', ...)
W-1004 (BOTH_CHILD_AND_ROOT): Models should not be a document root if they are in a layout box

# COVARIANTIE / CORRELATIE 
Olieprijzen en Voedselprijzen

In [143]:

y_axis = 'mp_price'


product_groups = df.groupby(['sub-region', 'adm0_name', 'cm_name', 'mortality_sum', 'frequency', 'rate', 'GDP', 'date'])[y_axis].mean().reset_index()

row1 = product_groups.loc[product_groups['adm0_name'] == 'Armenia']


prod = row1.pivot_table('mp_price', ['sub-region', 'adm0_name', 'date'], ['cm_name'])

new = prod.reset_index()
t = pd.merge(new, df[['adm0_name', 'date', 'mortality_sum', 'frequency', 'rate', 'GDP']], on=['adm0_name', 'date'])

result = t.corr(method='pearson')
print(result)

KeyError: 'GDP'

# Question 3
Can we see any correlations between other types of data, like: currency fluctuations, weather patterns, and/or refugee movements. Can we perhaps visualize refugee movements from the food price data? Do certain weather conditions influence market prices?

In [3]:
import pandas as pd
import os
import numpy as np

# get datasets
# https://data.world/unhcr/asylum-applications/workspace/file?filename=asylum_seekers_monthly_2016_12_08_152150.csv
asylum_file = 'asylum_seekers_monthly_2016_12_08_152150.csv'
df = pd.read_csv(asylum_file, low_memory=False)

food_prices_file = 'WFPVAM_FoodPrices_version5_Retail.csv'
df_food = pd.read_csv(food_prices_file)


####### pre-processing #######################################################################

df.columns = ['destination', 'origin', 'year', 'month', 'frequency']
# remove abundant rows
df.drop(df.index[:2], inplace=True)

# map month names to corresponding number to allow for convertion to daytime
MONTH_DICT = {'January': 1, 'February': 2, 'March': 3, 'April': 4, 'May': 5, 'June': 6, 
              'July': 7, 'August': 8, 'September': 9, 'October': 10, 'November': 11,
              'December': 12}

df['month'] = df['month'].map(MONTH_DICT)

# month and year column to single daytime column 'date'
df = df.assign(date=pd.to_datetime(df[['year', 'month']].assign(day=1)))
# drop now abundant columns month and year
df = df.drop('month', axis=1)
df = df.drop('year', axis=1)

# drop rows with undefined frequency
df = df.drop(df[df.frequency == '*'].index)
# also drop rows with unknown/various origins
df = df.drop(df[df.origin == 'Various/unknown'].index)

# NB: we could also drop the destination columns, since it only includes countries
# outside of our original dataset. But maybe it will allow for some dynamic hovering later on...
# so we'll keep it for now.

# see if different names are used for identical counrtries in the two databases:
# will end up with a list with countries that don't match; manually see if they should...
countries_food = df_food['Iadm0_name'].unique()
countries_refugee = df['origin'].unique()

possible_mismatches_1 = [country for country in countries_food 
                       if country not in countries_refugee]

possible_mismatches_2 = [country for country in countries_refugee
                        if country not in countries_food]

definite_mismatches = possible_mismatches_1 + possible_mismatches_2

mismatches = definite_mismatches.sort()
# print(mismatches)

# {'country name in refugee file': 'corresponding country name in food dataset'}
MISMATCH_DICT = {'Iran (Islamic Rep. of)': 'Iran  (Islamic Republic of)', 'Dem. Rep. of the Congo': 'Democratic Republic of the Congo',
                 'Central African Rep.': 'Central African Republic'}

# fix mis-matches
for (original, new) in MISMATCH_DICT.items():
    mask = df.origin == original
    df.loc[mask, 'origin'] = new
    


In [2]:
# reset index
df = df.reset_index(drop=True)
df.head()
###################################### pre-processing done ###################################


# set food_prices dataset to daytime to allow merging with refugee dataset
df_food['date'] = pd.to_datetime(df_food.date, format='%Y-%m')
# merge datasets 

new_df = pd.merge(df_food, df,  how='left', left_on=['Iadm0_name','date'], right_on = ['origin','date'])
# drop now abundantan origin row (equals Iadm0_name)
new_df[:50]



,Iadm0_name,mkt_name,cm_name,mp_price,date,um_name,Gradient,destination,origin,frequency
0,Afghanistan,Bamyan,Wheat,4483.786085,2004-04-01,KG,-195.204191,Australia,Afghanistan,3
1,Afghanistan,Bamyan,Wheat,4483.786085,2004-04-01,KG,-195.204191,Austria,Afghanistan,123
2,Afghanistan,Bamyan,Wheat,4483.786085,2004-04-01,KG,-195.204191,Belgium,Afghanistan,25
3,Afghanistan,Bamyan,Wheat,4483.786085,2004-04-01,KG,-195.204191,Bulgaria,Afghanistan,8
4,Afghanistan,Bamyan,Wheat,4483.786085,2004-04-01,KG,-195.204191,Canada,Afghanistan,8
5,Afghanistan,Bamyan,Wheat,4483.786085,2004-04-01,KG,-195.204191,Czech Rep.,Afghanistan,1
6,Afghanistan,Bamyan,Wheat,4483.786085,2004-04-01,KG,-195.204191,Cyprus,Afghanistan,5
7,Afghanistan,Bamyan,Wheat,4483.786085,2004-04-01,KG,-195.204191,Finland,Afghanistan,8
8,Afghanistan,Bamyan,Wheat,4483.786085,2004-04-01,KG,-195.204191,Denmark,Afghanistan,35
9,Afghanistan,Bamyan,Wheat,4483.786085,2004-04-01,KG,-195.204191,France,Afghanistan,13


In [16]:
# also drop rows with unknown/various origins
refugees = refugees.drop(refugees[refugees['Origin'] == 'Various/Unknown'].index)

In [17]:
print(refugees.head(10))

     Year Country / territory of asylum/residence  Origin  \
163  1960                  Dem. Rep. of the Congo  Angola   
179  1961                                 Burundi  Rwanda   
183  1961                  Dem. Rep. of the Congo  Angola   
184  1961                  Dem. Rep. of the Congo  Rwanda   
197  1961                                    Togo   Ghana   
200  1961                 United Rep. of Tanzania  Rwanda   
201  1961                                  Uganda  Rwanda   
204  1962                                 Burundi  Rwanda   
208  1962                  Dem. Rep. of the Congo  Angola   
209  1962                  Dem. Rep. of the Congo  Rwanda   

    Refugees (incl. refugee-like situations) Asylum-seekers (pending cases)  \
163                                   150000                            NaN   
179                                    30000                            NaN   
183                                   150000                            NaN   
184         

# QUESTION 4

In [103]:
y_axis = 'mp_price'


plots = []
group = df.groupby(['sub-region', 'adm0_name', 'cm_name', 'date'])[y_axis].mean().reset_index()
group = group.groupby('sub-region')

for gr1, row1 in group:
    print(gr1)
    if gr1 == 'Eastern Europe':
        print(gr1)
        product_groups = row1.groupby(['cm_name'])
        for gr3, row3 in product_groups:
            if 'Meat (beef)' in gr3:
                plot = figure(x_axis_label="Date", y_axis_label = "Price", x_axis_type="datetime", title="Avg Products in {}".format(gr3), toolbar_location="below", tools='hover,pan,wheel_zoom,box_zoom,reset')#, plot_width=2800, plot_height=2800)
                country_group = row3.groupby('adm0_name')
                for (gr2, row2), color in zip(country_group, colors):
                    row2_cds = ColumnDataSource(row2)
                    plot.line('date', y_axis, color=color, legend=gr2, source=row2_cds)
                plot.legend.click_policy = "hide"
                plot.legend.location = "top_right"
                hover = plot.select(dict(type=HoverTool))
                hover.tooltips = [
                    ("Product", "@cm_name"),
                    ("Value", "@Gradient"),
                    ]
                show(plot)
                plots.append(plot)
                break
        break

show(gridplot([plots]))

Central Asia
Eastern Europe
Eastern Europe


W-1004 (BOTH_CHILD_AND_ROOT): Models should not be a document root if they are in a layout box: Figure(id='c302fd90-07b6-44e3-a8b6-9aef96390bca', ...)
W-1004 (BOTH_CHILD_AND_ROOT): Models should not be a document root if they are in a layout box: Figure(id='84dc665e-d409-48d7-9c17-eaaadb5f885a', ...)
W-1004 (BOTH_CHILD_AND_ROOT): Models should not be a document root if they are in a layout box: Figure(id='d65c1eef-3615-451a-983d-b2f3446e3025', ...)
W-1004 (BOTH_CHILD_AND_ROOT): Models should not be a document root if they are in a layout box: Figure(id='4f251dc9-42a3-43cf-9959-af06ae93803f', ...)
W-1004 (BOTH_CHILD_AND_ROOT): Models should not be a document root if they are in a layout box: Figure(id='de0cd80d-1fbb-4fa2-a8ee-1bb3dff09255', ...)
W-1004 (BOTH_CHILD_AND_ROOT): Models should not be a document root if they are in a layout box: Figure(id='b4959177-0795-4f96-91e9-e6c0d00a1e6c', ...)
W-1004 (BOTH_CHILD_AND_ROOT): Models should not be a document root if they are in a layout box

In [101]:
y_axis = 'mp_price'
# df3 = tj.selecton_date(df2, '2003-01', '2005-01').copy()
dic = {'sub-region' : ['Eastern Europe'], tj.PROD : ['Meat (beef)', 'Meat (chicken, whole)', 'Meat (mixed, sausage)', 'Meat (pork)'], tj.COUNTRY: []}
prod = tj.df_pivot(df2, dic, tj.PRICE)
result = prod.corr(method='pearson')
print(result)

# product_groups = df.groupby(['sub-region', 'adm0_name', 'cm_name', 'date'])[y_axis].mean().reset_index()
# group = product_groups.groupby('adm0_name')

# for gr1, row1 in group:
# #         date_range = row1[(row1['date'] >= '2016-01-01')]# & (row1['date'] <= '2017-01-01')]
#     prod = row1.pivot_table('mp_price', ['adm0_name', 'date'], 'cm_name')
#     result = prod.corr(method='pearson')
    
#     print(gr1)
#     print(result)
# #         print(result.groupby(['Fuel (diesel)', 'Fuel (petrol-gasoline)']).mean())



Info                                                Eastern Europe - Meat (beef) - Ukraine -   \
Info                                                                                            
Eastern Europe - Meat (beef) - Ukraine -                                             1.000000   
Eastern Europe - Meat (chicken, whole) - Ukrain...                                   0.946048   
Eastern Europe - Meat (mixed, sausage) - Ukrain...                                   0.962734   
Eastern Europe - Meat (pork) - Ukraine -                                             0.981646   

Info                                                Eastern Europe - Meat (chicken, whole) - Ukraine -   \
Info                                                                                                      
Eastern Europe - Meat (beef) - Ukraine -                                                     0.946048     
Eastern Europe - Meat (chicken, whole) - Ukrain...                                           1.0